In [2]:
%matplotlib notebook
# Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
# Load in csv
defunciones_2012_df = pd.read_csv("defunciones.csv")
defunciones_2013_df = pd.read_csv("defunciones.csv")
defunciones_2014_df = pd.read_csv("defunciones.csv")
defunciones_2015_df = pd.read_csv("defunciones.csv")
defunciones_2016_df = pd.read_csv("defunciones.csv")
defunciones_2017_df = pd.read_csv("defunciones.csv")

defunciones_2012_df.head()

,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,...,anio_cert,maternas,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax
0,1,1,1,1,15,1,1,1,15,1,...,2011,NaN,9,1,88,88,888,8888,0,999
1,1,1,1,2,1,7777,1,1,15,1,...,2011,NaN,9,1,88,88,888,8888,0,999
2,1,1,1,1,1,7777,1,1,1,7777,...,2012,NaN,9,1,88,88,888,8888,0,999
3,1,1,1,5,1,106,1,1,15,1,...,2012,NaN,9,1,88,88,888,8888,0,999
4,1,1,1,1,15,1,1,1,15,1,...,2012,NaN,9,1,88,88,888,8888,0,999
